In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
import time
import requests
import pandas as pd

api_key = os.environ.get('riot_api')

def get_puuid_by_Id(summonerId=None):
    root_url = 'https://na1.api.riotgames.com/tft/summoner/v1/summoners/'
    link = f'{root_url}{summonerId}?api_key={api_key}'
    response = requests.get(link)
    return response.json()['puuid']

def get_name_and_tag(puuid=None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'
    response = requests.get(link)

    id = {
        'gameName' : response.json()['gameName'],
        'tagLine' : response.json()['tagLine']
    }
    

    return id['gameName']

def get_ladder():
    root = 'https://na1.api.riotgames.com/'
    chall = 'tft/league/v1/challenger?queue=RANKED_TFT'
    gm = 'tft/league/v1/grandmaster?queue=RANKED_TFT'
    masters = 'tft/league/v1/master?queue=RANKED_TFT'

    chall_response = requests.get(root + chall + '&api_key=' + api_key)
    gm_response = requests.get(root + gm + '&api_key=' + api_key)
    masters_response = requests.get(root + masters + '&api_key=' + api_key)

    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    masters_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, masters_df]).reset_index(drop=True)[:10]
    ladder = ladder.reset_index().drop(columns=['rank']).rename(columns={'index':'rank'})
    ladder['rank']+=1
    return ladder

def get_ladder_ids():
    root = 'https://na1.api.riotgames.com/'
    chall = 'tft/league/v1/challenger?queue=RANKED_TFT'
    gm = 'tft/league/v1/grandmaster?queue=RANKED_TFT'
    masters = 'tft/league/v1/master?queue=RANKED_TFT'

    chall_response = requests.get(root + chall + '&api_key=' + api_key)
    gm_response = requests.get(root + gm + '&api_key=' + api_key)
    masters_response = requests.get(root + masters + '&api_key=' + api_key)

    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    masters_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, masters_df]).reset_index(drop=True)[:50]
    ladder = ladder.reset_index().drop(columns=['rank']).rename(columns={'index':'rank'})
    ladder['rank']+=1
    return ladder['summonerId']

In [2]:
ladder_ids = get_ladder_ids()
ladder_ids

0       E-Lh7wNMhUwm2jgls39dEDDjtQalOImj8Tdq7S7VMqecir0
1       RBdJQzZvbAaaM4zKInCZo_RK1-1ZGIxdpM9LvDS59FAO-VE
2     cGJmgIyHptUPfJFc_SUrCRtS1pT6Gy43Cp0hIHFVzo55Nd...
3      ZupKjzSOYY6Rb5Ml_x98aVdxABJDcFh8IADIdvlXwcnakRfE
4       txFCeY8hxjFOn9m_hj1M5VbypwRVHoZ5DWuATGyfXFJw6S4
5       zNCQobM_UBWbQFM9oqufdm8linK65t5KStQNkyscRWaQ284
6       KUQBy-ZOMPu9MNl6A5opUNusxa0_whCTkmOQMLM-efFtpTo
7      opfTl4Ow5hNwCfFUOnovHtem-JkBTw8T-WEIIAKJDD2hkoU8
8       jg2BhfqHqc6EDNZOdUVFGq5LbnOqff7hlBnn5FQthqEcYNY
9     tPZ6tT2NT_UQUpg56WrX3MZzM_Dq_ncsCOwehCjCGITZHw...
10      0d-qLpIefAzbYIFBSoMqL82nRKFD61KURrrqynaYmkOGYuM
11      Ujl8-NrpnNWFcLNgJ35SFWJ788qEBcFJHhRr5EpUyLueu58
12      11phZfcNTxwyuCo-hhK2HShp66PMCe3k_xFzc-qjAGajNmk
13     pcvP1tgxvmsZL40bo-f3Cxx7I3cCXas1gNHFFwXfDDMuVo3U
14    KV2fU1m23fGsqY6cdlNBC5yj7bOKMXt3oaC0eSFxI_s6Gz...
15     1o9ihEEUVVNE5oNTBnylgnzzTJ-xNL7lYqAw2akvKar4XR41
16      Yqg4erLJwBqVB2hJkO_Jhia34fD6EkdR3xXwfrosXwYLtbE
17      h2-oVPtp1NyjRHzrWIVhYgtiQIthNEVu_ZNLDw7U

In [3]:
puuid_df = pd.DataFrame()
for summoner_id in ladder_ids:
    puuid_list = pd.DataFrame({'puuid': [get_puuid_by_Id(summonerId=summoner_id)]})
    
    puuid_df = pd.concat([puuid_df, puuid_list]).reset_index(drop=True)
puuid_list = puuid_df['puuid']
puuid_df

,puuid
0,1lniY8dPYKom8GPmoOrsR63qHO81VVHla4tM359WKpqZei...
1,TE7A88gc7fRfHc6r7rn7eU7CSBUAxT0bvGB269hN4FvbdN...
2,F_EXxdYtWONatqKsZJsHY6CgBv7sgFLsOic98m4rDUOlAk...
3,S6bJo296D_Bru7kAmQHl6dI0OHHxzGCMHxRq4fXrbePCD7...
4,x-GiIxJ5muZFPFOXbxbjim7EZgfLq4slRHnqzsl2QEYj_0...
5,xQui9qXUi2g6q6qy3m_Hh84Ep_QUlprL3BE6DNc-cu_5LH...
6,gW7MdNAHhk4MTxuVqV1jlR4kuUPdwrAkD-lTgKjitXGXNu...
7,RJdrv5h6TMCgA2eQkNVl4KtoxvlLj79R4QezPAY68PVmkm...
8,Tdm0rhtJhVfxyEPdo2MIMIIW2VlREbntO-pAgJyN4yRh3A...
9,zeh3osQ9J5WWR-iNvDRb_oabWXb_FXU7ipVdmjBVZCwdHo...


In [4]:
names_df = pd.DataFrame()
for puuids in puuid_list:
    names_list = pd.DataFrame({'summonernames' : [get_name_and_tag(puuid=puuids)]})
    names_df = pd.concat ([names_df, names_list]).reset_index(drop=True)
names_df

,summonernames
0,torontokyoArkjow
1,robinsongz
2,TLeyds
3,Dishsoap
4,VIT Milk
5,TL grea
6,Ripple Overdrive
7,FNCwasianiverson
8,Nik
9,VOIDS1N


In [5]:
def get_match(puuid=None, start=0, count=20):
    root = 'https://americas.api.riotgames.com/'
    endpoint = f'/tft/match/v1/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}'

    response = requests.get(root + endpoint + query_params + '&api_key=' + api_key)

    return response.json()

def get_match_data(match_id=None):
    root = 'https://americas.api.riotgames.com/'
    endpoint = f'/tft/match/v1/matches/{match_id}'

    response = requests.get(root + endpoint + '?api_key=' + api_key)
    return response.json()

In [6]:
puuid='VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w'
def process_matches(matchjson, puuid):
    arcana = 0
    chrono = 0
    dragon = 0
    druid = 0
    eldritch = 0 
    faerie = 0 
    frost = 0 
    honeymancy = 0 
    portal = 0 
    pyro = 0 
    ravenous = 0 
    sugarcraft = 0 
    witchcraft = 0 
    ascendant = 0 
    bastion = 0 
    batqueen = 0 
    blaster = 0 
    explorer = 0 
    hunter = 0 
    incantor = 0 
    mage = 0 
    multistriker = 0 
    preserver = 0 
    scholar = 0 
    shapeshifter = 0 
    vanguard = 0 
    warrior = 0 

    metadata = matchjson['metadata']
    info = matchjson['info']

    match_id = metadata['match_id']
    participants = metadata['participants']

    name_list = []
    for i in range (len(participants)):
        name = get_name_and_tag(puuid=participants[i])
        name_list.append(name)

    players = info['participants']
    game_creation = info['game_datetime']
    game_length = info['game_length']
    game_version = info['game_version']
    tft_set_number = info['tft_set_number']

    player = players[participants.index(puuid)]

    level = player['level']
    placement = player['placement']
    last_round = player['last_round']
    time_eliminated = player['time_eliminated']
    total_damage_to_players = player['total_damage_to_players']
    players_eliminated = player['players_eliminated']

    traits = player['traits']
    units = player['units']
    for trait in traits:
        if trait['name'] == 'TFT12_Arcana':
            arcana = trait['num_units']
        if trait['name'] == 'TFT12_Chrono':
            chrono = trait['num_units']
        if trait['name'] == 'TFT12_Dragon':
            dragon = trait['num_units']
        if trait['name'] == 'TFT12_Druid':
            druid = trait['num_units']
        if trait['name'] == 'TFT12_Eldritch':
            eldritch = trait['num_units']
        if trait['name'] == 'TFT12_Faerie':
            faerie = trait['num_units']
        if trait['name'] == 'TFT12_Frost':
            frost = trait['num_units']
        if trait['name'] == 'TFT12_Honeymancy':
            honeymancy = trait['num_units']
        if trait['name'] == 'TFT12_Portal':
            portal = trait['num_units']
        if trait['name'] == 'TFT12_Pyro':
            pyro = trait['num_units']
        if trait['name'] == 'TFT12_Ravenous':
            ravenous = trait['num_units']
        if trait['name'] == 'TFT12_Sugarcraft':
            sugarcraft = trait['num_units']
        if trait['name'] == 'TFT12_Witchcraft':
            witchcraft = trait['num_units']
        if trait['name'] == 'TFT12_Ascendant':
            ascendant = trait['num_units']
        if trait['name'] == 'TFT12_Bastion':
            bastion = trait['num_units']
        if trait['name'] == 'TFT12_BatQueen':
            batqueen = trait['num_units']
        if trait['name'] == 'TFT12_Blaster':
            blaster = trait['num_units']
        if trait['name'] == 'TFT12_Explorer':
            explorer = trait['num_units']
        if trait['name'] == 'TFT12_Hunter':
            hunter = trait['num_units']
        if trait['name'] == 'TFT12_Incantor':
            incantor = trait['num_units']
        if trait['name'] == 'TFT12_Mage':
            mage = trait['num_units']
        if trait['name'] == 'TFT12_Multistriker':
            multistriker = trait['num_units']
        if trait['name'] == 'TFT12_Preserver':
            preserver = trait['num_units']
        if trait['name'] == 'TFT12_Scholar':
            scholar = trait['num_units']
        if trait['name'] == 'TFT12_Shapeshifter':
            shapeshifter = trait['num_units']
        if trait['name'] == 'TFT12_Vanguard':
            vanguard = trait['num_units']
        if trait['name'] == 'TFT12_Warrior':
            warrior = trait['num_units']
    matchDF = pd.DataFrame({
        'match_id' : [match_id],
        'participants' : [name_list],
        'game_creation' : [game_creation],
        'game_length' : [game_length],
        'game_version' : [game_version],
        'tft_set_number' : [tft_set_number],
        'puuid' : [puuid],
        'summoner_name' : [get_name_and_tag(puuid=puuid)],
        'level' : [level],
        'placement' : [placement],
        'last_round' : [last_round],
        'time_eliminated' : [time_eliminated],
        'total_damage_to_players' : [total_damage_to_players],
        'players_eliminated' : [players_eliminated],
        'arcana' : [arcana],  
        'chrono' : [chrono],  
        'dragon' : [dragon],  
        'druid' : [druid],  
        'eldritch' : [eldritch],   
        'faerie' : [faerie],   
        'frost' : [frost],   
        'honeymancy' : [honeymancy],   
        'portal' : [portal],   
        'pyro' : [pyro],   
        'ravenous' : [ravenous],   
        'sugarcraft' : [sugarcraft],  
        'witchcraft' : [witchcraft],  
        'ascendant' : [ascendant],  
        'bastion' : [bastion],  
        'batqueen' : [batqueen],  
        'blaster' : [blaster],  
        'explorer' : [explorer],  
        'hunter' : [hunter],  
        'incantor' : [incantor],  
        'mage' : [mage],  
        'multistriker' : [multistriker],  
        'preserver' : [preserver],  
        'scholar' : [scholar],  
        'shapeshifter' : [shapeshifter],  
        'vanguard' : [vanguard],  
        'warrior' : [warrior] 
    })
    
    return matchDF

In [7]:
get_name_and_tag(puuid=puuid)

'VIT k3soju'

In [15]:
match_ids = get_match(puuid=puuid, start=0, count=100)

df = pd.DataFrame()
for match_id in match_ids:
    game = get_match_data(match_id)
    matchDF = process_matches(game, puuid)

    df = pd.concat([df, matchDF])
    time.sleep(24)
df

,match_id,participants,game_creation,game_length,game_version,tft_set_number,puuid,summoner_name,level,placement,...,explorer,hunter,incantor,mage,multistriker,preserver,scholar,shapeshifter,vanguard,warrior
0,NA1_5129922204,"[Ren Hao Jiang, Euphorian, Lab 003 Broseph, 2 ...",1728501904351,2025.928833,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,6,...,0,0,0,0,1,1,2,0,2,0
0,NA1_5129903699,"[sirmasterapu, VIT k3soju, Lab 003 Broseph, 14...",1728499733528,2110.918701,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,10,2,...,0,6,0,0,0,0,0,4,0,0
0,NA1_5129896111,"[PestControl, IDxmo, VIT k3soju, techzz, keeso...",1728497477474,2161.710205,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,2,...,1,0,0,7,0,0,0,0,2,0
0,NA1_5129884902,"[Pneuma, hsk, PestControl, Estimate, VIT k3soj...",1728495017252,2081.028809,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,4,...,1,0,0,7,0,0,0,0,2,0
0,NA1_5129863699,"[meowmeowmeow, Sylvanas327, Lesseraqueen, VIT ...",1728492689862,2077.107910,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,9,4,...,0,6,0,0,0,0,0,4,0,0
0,NA1_5129859724,"[imriveting, L0VE D1VE, otter popped, mrotdog,...",1728491047628,2191.202148,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,7,...,0,0,0,0,1,1,2,0,2,0
0,NA1_5129851808,"[otter popped, ArtemisOfTheMoon, wigwugg, EMIT...",1728488628655,2140.206543,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,9,4,...,0,0,0,7,0,0,0,1,2,0
0,NA1_5129845801,"[TimTam Bikkies, L0VE D1VE, Wet Jungler, EvilH...",1728486226843,2310.405273,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,3,...,0,0,0,7,1,0,0,0,2,0
0,NA1_5129837904,"[NEWJEANSMINJI, Lab 003 Broseph, Slooper, Xend...",1728483659215,2366.933594,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,9,2,...,0,0,0,7,1,0,0,0,2,0
0,NA1_5129828337,"[mrotdog, eudaymonia, Emailsupport1234, Xender...",1728481371414,2118.054688,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,5,...,1,0,0,7,0,0,0,0,2,0


In [16]:
df = df.reset_index(drop=True)

In [17]:
df['uuid'] = df['match_id'] + '_' + df['puuid']
df = df.set_index('uuid')
df

,match_id,participants,game_creation,game_length,game_version,tft_set_number,puuid,summoner_name,level,placement,...,explorer,hunter,incantor,mage,multistriker,preserver,scholar,shapeshifter,vanguard,warrior
uuid,,,,,,,,,,,,,,,,,,,,,
NA1_5129922204_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129922204,"[Ren Hao Jiang, Euphorian, Lab 003 Broseph, 2 ...",1728501904351,2025.928833,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,6,...,0,0,0,0,1,1,2,0,2,0
NA1_5129903699_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129903699,"[sirmasterapu, VIT k3soju, Lab 003 Broseph, 14...",1728499733528,2110.918701,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,10,2,...,0,6,0,0,0,0,0,4,0,0
NA1_5129896111_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129896111,"[PestControl, IDxmo, VIT k3soju, techzz, keeso...",1728497477474,2161.710205,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,2,...,1,0,0,7,0,0,0,0,2,0
NA1_5129884902_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129884902,"[Pneuma, hsk, PestControl, Estimate, VIT k3soj...",1728495017252,2081.028809,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,4,...,1,0,0,7,0,0,0,0,2,0
NA1_5129863699_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129863699,"[meowmeowmeow, Sylvanas327, Lesseraqueen, VIT ...",1728492689862,2077.107910,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,9,4,...,0,6,0,0,0,0,0,4,0,0
NA1_5129859724_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129859724,"[imriveting, L0VE D1VE, otter popped, mrotdog,...",1728491047628,2191.202148,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,7,...,0,0,0,0,1,1,2,0,2,0
NA1_5129851808_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129851808,"[otter popped, ArtemisOfTheMoon, wigwugg, EMIT...",1728488628655,2140.206543,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,9,4,...,0,0,0,7,0,0,0,1,2,0
NA1_5129845801_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129845801,"[TimTam Bikkies, L0VE D1VE, Wet Jungler, EvilH...",1728486226843,2310.405273,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,8,3,...,0,0,0,7,1,0,0,0,2,0
NA1_5129837904_VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4opIoD3KK04aqkIUJMB4AF8DkjJwT4zA5w,NA1_5129837904,"[NEWJEANSMINJI, Lab 003 Broseph, Slooper, Xend...",1728483659215,2366.933594,Linux Version 14.20.623.6777 (Oct 03 2024/15:4...,12,VeXtOJDJV2Zvx1E_43E3itOp8uaKIVeABXDOGxT48uQC4o...,VIT k3soju,9,2,...,0,0,0,7,1,0,0,0,2,0


In [18]:
from pangres import upsert
from sqlalchemy import text, create_engine

db_username = os.environ.get('db_username')
db_password = os.environ.get('db_password')
db_host = os.environ.get('db_host')
db_port = os.environ.get('db_port')
db_name = os.environ.get('db_name')

def create_db_connection_string(db_username, db_password, db_host, db_port, db_name):
    connection_url = 'postgresql+psycopg2://' + db_username + ':' + db_password + '@' + db_host + ':' + db_port + '/' + db_name
    return connection_url

connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)

db_engine = create_engine(connection_url, pool_recycle=3600)

connection = db_engine.connect()

In [19]:
upsert(con=connection, df=df, schema='soloq', table_name='regional_player_matches', create_table=True, create_schema=True, if_row_exists='update')

In [20]:
connection.commit()